<a href="https://colab.research.google.com/github/FelipeGadelha18/Google-Colab/blob/main/Teste_Intuitive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:

!pip install PyPDF2 pandas

import PyPDF2
import pandas as pd
from collections import defaultdict
from google.colab import files
import os


uploaded = files.upload()
file_name = list(uploaded.keys())[0]

mapeamento_abreviacoes = {
    'OD': 'Seguro Odontológico',
    'AMB': 'Seguro Ambulatorial',
    'HCO': 'Seguro Hospitalar Com Obstetrícia',
    'HSO': 'Seguro Hospitalar Sem Obstetrícia',
    'REF': 'Plano Referência',
    'PAC': 'Procedimento de Alta Complexidade',
    'DUT': 'Diretriz de Utilização'
}

def extract_procedures_data(pdf_file):
    data = defaultdict(list)

    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)

        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text()

            lines = text.split('\n')

            if "PROCEDIMENTO" in lines[0] or "PROCEDIMENTO" in lines[1]:
                start_idx = 2 if "PROCEDIMENTO" in lines[0] else 3

                for line in lines[start_idx:]:
                    if any(x in line for x in ['OD:', 'AMB:', 'HCO:', 'HSO:', 'REF:', 'PAC:', 'DUT:']):
                        continue

                    parts = line.split()

                    if len(parts) >= 12:
                        data['PROCEDIMENTO'].append(' '.join(parts[:-11]))
                        data['RN (alteração)'].append(parts[-11])
                        data['VIGÊNCIA'].append(parts[-10])
                        data['Modalidade OD'].append(parts[-9])
                        data['Modalidade AMB'].append(parts[-8])
                        data['Modalidade HCO'].append(parts[-7])
                        data['Modalidade HSO'].append(parts[-6])
                        data['Tipo REF'].append(parts[-5])
                        data['Complexidade PAC'].append(parts[-4])
                        data['Diretriz DUT'].append(parts[-3])
                        data['SUBGRUPO'].append(parts[-2])
                        data['GRUPO'].append(parts[-1])
                        data['CAPÍTULO'].append('PROCEDIMENTOS CIRÚRGICOS E INVASIVOS')

    return pd.DataFrame(data)


df = extract_procedures_data(file_name)


for col, abrev in [('Modalidade OD', 'OD'), ('Modalidade AMB', 'AMB'),
                   ('Modalidade HCO', 'HCO'), ('Modalidade HSO', 'HSO'),
                   ('Tipo REF', 'REF'), ('Complexidade PAC', 'PAC'),
                   ('Diretriz DUT', 'DUT')]:
    df[col] = df[col].map(lambda x: mapeamento_abreviacoes[abrev] if x == abrev else x)


col_order = [
    'PROCEDIMENTO', 'RN (alteração)', 'VIGÊNCIA', 'SUBGRUPO', 'GRUPO', 'CAPÍTULO',
    'Modalidade OD', 'Modalidade AMB', 'Modalidade HCO', 'Modalidade HSO',
    'Tipo REF', 'Complexidade PAC', 'Diretriz DUT'
]
df = df[col_order]

csv_path = 'procedimentos_saude_formatado.csv'
df.to_csv(csv_path, sep=';', decimal=',', encoding='utf-8-sig', index=False, quoting=1)

if os.path.exists(csv_path):
    print(f"Arquivo {csv_path} criado com sucesso!")
    print("Tamanho do arquivo:", os.path.getsize(csv_path), "bytes")

    files.download(csv_path)
else:
    print("Erro: O arquivo CSV não foi criado corretamente.")

print("\nPrévia dos dados (primeiras 5 linhas):")
print(df.head())

Saving Anexo_I.pdf to Anexo_I (6).pdf
Arquivo procedimentos_saude_formatado.csv criado com sucesso!
Tamanho do arquivo: 565557 bytes


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Prévia dos dados (primeiras 5 linhas):
                                        PROCEDIMENTO RN (alteração)  VIGÊNCIA  \
0                          RN nº 603/2024, vigente a         partir        de   
1  ATENDIMENTO/ACOMPANHAMENTO EM HOSPITAL-DIA PSI...           (COM  DIRETRIZ   
2              ATENDIMENTO INTEGRAL AO RECÉM-NASCIDO          (SALA        DE   
3                 ATENDIMENTO MÉDICO DO INTENSIVISTA             EM       UTI   
4                                        ATENDIMENTO     PEDIÁTRICO         A   

       SUBGRUPO       GRUPO                              CAPÍTULO  \
0            de  07/05/2024  PROCEDIMENTOS CIRÚRGICOS E INVASIVOS   
1  HOSPITALARES          OU  PROCEDIMENTOS CIRÚRGICOS E INVASIVOS   
2  HOSPITALARES          OU  PROCEDIMENTOS CIRÚRGICOS E INVASIVOS   
3  HOSPITALARES          OU  PROCEDIMENTOS CIRÚRGICOS E INVASIVOS   
4  HOSPITALARES          OU  PROCEDIMENTOS CIRÚRGICOS E INVASIVOS   

  Modalidade OD Modalidade AMB                     Modalid